In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [13]:
# data
ipl_data = {
    "Team": [
        "Riders",
        "Riders",
        "Devils",
        "Devils",
        "Kings",
        "Kings",
        "Kings",
        "Kings",
        "Riders",
        "Royals",
        "Royals",
        "Riders",
    ],
    "Rank": [1, 2, 2, 3, 3, 4, 1, 1, 2, 4, 1, 2],
    "Year": [2014, 2015, 2014, 2015, 2014, 2015, 2016, 2017, 2016, 2014, 2015, 2017],
    "Point": [876, 789, 863, 673, 741, 812, 756, 788, 694, 701, 804, 690]
}
df = DataFrame(ipl_data)
df

,Team,Rank,Year,Point
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


### Groupby

In [11]:
# df.groupby("묶음 기준이 되는 컬럼")["적용받는 컬럼"]."적용받는 연산"
df.groupby("Team")["Point"].std()

Team
Devils    134.350288
Kings      31.899582
Riders     88.567771
Royals     72.831998
Name: Point, dtype: float64

In [12]:
# 여러 column으로 묶을 수 있음
h_index = df.groupby(["Team", "Year"])["Point"].sum()
h_index

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2015    812
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
Name: Point, dtype: int64

In [14]:
h_index.index

MultiIndex([('Devils', 2014),
            ('Devils', 2015),
            ( 'Kings', 2014),
            ( 'Kings', 2015),
            ( 'Kings', 2016),
            ( 'Kings', 2017),
            ('Riders', 2014),
            ('Riders', 2015),
            ('Riders', 2016),
            ('Riders', 2017),
            ('Royals', 2014),
            ('Royals', 2015)],
           names=['Team', 'Year'])

In [15]:
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,812.0,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN


In [16]:
h_index.unstack().stack()

Team    Year
Devils  2014    863.0
        2015    673.0
Kings   2014    741.0
        2015    812.0
        2016    756.0
        2017    788.0
Riders  2014    876.0
        2015    789.0
        2016    694.0
        2017    690.0
Royals  2014    701.0
        2015    804.0
dtype: float64

In [17]:
h_index.reset_index()

,Team,Year,Point
0,Devils,2014,863
1,Devils,2015,673
2,Kings,2014,741
3,Kings,2015,812
4,Kings,2016,756
5,Kings,2017,788
6,Riders,2014,876
7,Riders,2015,789
8,Riders,2016,694
9,Riders,2017,690


In [18]:
# index1, 2 switch
h_index.swaplevel()

Year  Team  
2014  Devils    863
2015  Devils    673
2014  Kings     741
2015  Kings     812
2016  Kings     756
2017  Kings     788
2014  Riders    876
2015  Riders    789
2016  Riders    694
2017  Riders    690
2014  Royals    701
2015  Royals    804
Name: Point, dtype: int64

In [23]:
h_index.swaplevel().sort_index(level=0)

Year  Team  
2014  Devils    863
      Kings     741
      Riders    876
      Royals    701
2015  Devils    673
      Kings     812
      Riders    789
      Royals    804
2016  Kings     756
      Riders    694
2017  Kings     788
      Riders    690
Name: Point, dtype: int64

### Grouped
* Groupby에 의해 Split된 상태를 추출 가능
* 추출된 group 정보에는 세 가지 유형의 apply가 가능
* `Aggregation` : 요약된 통계정보를 추출
* `Transformation` : 해당 정보를 변환
* `Filtration` : 특정 정보를 제거 하여 보여주는 필터링 기능

In [25]:
grouped = df.groupby("Team")
for name, group in grouped:
    print(name)
    print(group)

Devils
     Team  Rank  Year  Point
2  Devils     2  2014    863
3  Devils     3  2015    673
Kings
    Team  Rank  Year  Point
4  Kings     3  2014    741
5  Kings     4  2015    812
6  Kings     1  2016    756
7  Kings     1  2017    788
Riders
      Team  Rank  Year  Point
0   Riders     1  2014    876
1   Riders     2  2015    789
8   Riders     2  2016    694
11  Riders     2  2017    690
Royals
      Team  Rank  Year  Point
9   Royals     4  2014    701
10  Royals     1  2015    804


In [29]:
grouped.get_group("Devils")

,Team,Rank,Year,Point
2,Devils,2,2014,863
3,Devils,3,2015,673


### Groupby - aggregation

In [30]:
grouped.agg(sum)

,Rank,Year,Point
Team,,,
Devils,5,4029,1536
Kings,9,8062,3097
Riders,7,8062,3049
Royals,5,4029,1505


In [31]:
grouped.agg(np.mean)

,Rank,Year,Point
Team,,,
Devils,2.50,2014.5,768.00
Kings,2.25,2015.5,774.25
Riders,1.75,2015.5,762.25
Royals,2.50,2014.5,752.50


In [32]:
grouped["Point"].agg([np.sum, np.mean, np.std])

,sum,mean,std
Team,,,
Devils,1536,768.00,134.350288
Kings,3097,774.25,31.899582
Riders,3049,762.25,88.567771
Royals,1505,752.50,72.831998


### Groupby - transformation
* Aggregation과 달리 Key값 별로 요약된 정보가 아님
* 개별 데이터의 변환을 지원

In [33]:
df

,Team,Rank,Year,Point
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [35]:
# max나 min처럼 Series 데이터에 적용되는 데이터들은 Key값을 기준으로 Grouped된 데이터 기준
score = lambda x: (x.max())
grouped.transform(score)

,Rank,Year,Point
0,2,2017,876
1,2,2017,876
2,3,2015,863
3,3,2015,863
4,4,2017,812
5,4,2017,812
6,4,2017,812
7,4,2017,812
8,2,2017,876
9,4,2015,804


In [36]:
# 정규화
score = lambda x: (x - x.mean()) / x.std()
grouped.transform(score)

,Rank,Year,Point
0,-1.500000,-1.161895,1.284327
1,0.500000,-0.387298,0.302029
2,-0.707107,-0.707107,0.707107
3,0.707107,0.707107,-0.707107
4,0.500000,-1.161895,-1.042333
5,1.166667,-0.387298,1.183401
6,-0.833333,0.387298,-0.572108
7,-0.833333,1.161895,0.431040
8,0.500000,0.387298,-0.770596
9,0.707107,-0.707107,-0.707107


### Groupby - Filter

In [37]:
# filter안에는 boolean 조건이 존재해야 함
# len(x)는 grouped된 dataframe의 개수
df.groupby("Team").filter(lambda x: len(x) >= 3)  # 팀별로 3개 이상의 데이터가 있는 애들만 출력해라

,Team,Rank,Year,Point
0,Riders,1,2014,876
1,Riders,2,2015,789
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
11,Riders,2,2017,690


In [39]:
df.groupby("Team").filter(lambda x: x["Rank"].sum() > 2)

,Team,Rank,Year,Point
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [40]:
df.groupby("Team").filter(lambda x: x["Point"].sum() > 1000)

,Team,Rank,Year,Point
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701
